# Introduction to Q#
### Unitary Fund Group meeting

Dr. Sarah Kaiser |  [@crazy4pi314](twitter.com/crazy4pi314) |  22 Oct 2020

---
    
  

## 📃Agenda
 
1. Give you a tour of Q# and why I am using it for research, 
2. show you a case study of using Q# to do research on qRAM, and
3. take a look at QIR (quantum intermediate representation) proposal!


# 📊 What do you know about Q#?

# Q\# : Microsoft's quantum programming language

- Open source language domain-specific to quantum computing
    - Strongly Typed
    - Immutable by default
- Used with the [_Quantum Development Kit_](https://www.microsoft.com/en-us/quantum/development-kit) which provides lots of tools for writing and running your programs.
- Designed to be integrated with a number of languages/platforms like Python and .NET

<center>
    <img src="media/stack.png" width="80%">
</center>



## Design goals:

**tl;dr:** Because we want to write algorithms, not circuits.

### Q\# Design Requirements:
1. Algorithms must be expressed in terms of abstract qubits, rather than physical qubits. 
2. Algorithms need to allow integrated quantum and classical computation. 
3. Higher-order protocols such as phase estimation and oblivious amplitude amplification must be expressible.
4. Higher-order transformations such as taking the adjoint of an operation must be natively expressible.
6. Tasks such as gate synthesis, gate sequence optimization, and ancilla management should be delegated to the compiler. 
7. Algorithms should be required to respect the laws of physics.

### Design Principles:
1. Start minimal and evolve carefully based on user experience.
2. Be quantum first, familiar second.
3. Use libraries wherever possible, rather than language features.
4. Keep clear, well-defined semantics to enable rich optimizations and transformations in the compiler back-end.

## What do I write in a Q# program?

- **Functions**: `Sin`, `ln`, reversing arrays, etc.
    - Deterministic actions, similar to mathematical definition for functions
- **Operations**: Everything else 😁 
    - Working with qubits is always an operation
- _User defined Types_

## Q# Hello World

- The `:` here shows you what type an variable or return of a function or operation is
- Everything in Q# is `()` in, `()` out

In [ ]:
function Greeting(name : String) : Unit {
    Message($"Hello World! Nice to meet you {name} 💖");
}

In [ ]:
%simulate Greeting name="Sarah"

## Q# Hello _Quantum_ world

In [ ]:
open Microsoft.Quantum.Measurement;

In [ ]:
operation Qrng() : Result {
    using (qubit = Qubit()) {   // Preparing the qubit
        H(qubit);               // Do operation H
        return MResetZ(qubit);  // Measure and reset qubit
    }
}

In [ ]:
%simulate Qrng

## Functions + Types in Q#

- Functions are ways to define deterministic, classical calculations.
- Q# is a strongly typed language, with a variety of built-in types and ways for you to define your own!

In [ ]:
// Library has the definition of the Complex type we use below.
open Microsoft.Quantum.Math;

In [ ]:
// A user defined type for an array of complex numbers.
newtype ComplexArray = (Count : Int, Values : Complex[]);

In [ ]:
// Converts an array of real numbers to a ComplexArray
function AsComplexArray(data : Double[]) : ComplexArray {

    // Start with a new ComplexArray to put your results in
    mutable results = ComplexArray(0, new Complex[0]);

    for (item in data) {
        // update-and-reassign statement
        set results w/= Values <- results::Values + [Complex(item, 0.)]; 
    }
    
    // Return the results with the Count named item filled out
    return results w/ Count <- Length(results::Values); 
}

function ConvertedData() : ComplexArray {
    let data = [1.0, 2.0, 3.0];
    return AsComplexArray(data);
}

In [ ]:
%simulate ConvertedData

## `using` Qubits in Q#

- Qubits are a resource that are requested from the runtime when you need them and returned when you are done.
- No distinction on physical/logical
- Can also `borrow` qubits

In [ ]:
operation Qrng() : Result {
    using (qubit = Qubit()) {   // Preparing the qubit
        H(qubit);               // Do operation H
        return MResetZ(qubit);  // Measure and reset qubit
    }
}

## Operations are First Class 🥂
- Operations and functions in Q# are first-class entities; they can be passed to other operations, assigned to variables, and used like any other value

In [ ]:
operation ApplyTwice(op : (Qubit => Unit), target : Qubit) : Unit {
    op(target);
    op(target);
}

function SquareOperation(op : (Qubit => Unit)) : (Qubit => Unit) {
    return ApplyTwice(op, _);
}

## Generating new operations in Q# ##

The _functors_ `Adjoint` and `Controlled` allow you to generate new operations without changes to your code to implement those versions.

In [ ]:
operation ApplyMultiControlledNOT(control: Qubit[], target : Qubit) : Unit {
    // Apply X normally:
    // X(target);
    
    Controlled X(control, target);
}

In [ ]:
open Microsoft.Quantum.Diagnostics;

operation UseCtlFunctor() : Unit {
    using ((controls, target) = (Qubit[2], Qubit())) {
    
        // Preparing uniform superposition
        ApplyToEach(H, controls);
        
        // Apply our controlledNOT
        ApplyMultiControlledNOT(controls, target);
        
        // Print simulator diagnostics
        DumpMachine();
        ResetAll(controls + [target]);
    }  
}

### Q# programs can be run from:

- the [command line](https://docs.microsoft.com/en-us/quantum/user-guide/host-programs?tabs=tabid-python#q-from-the-command-prompt), if built as stand-alone applications
- [Python](https://docs.microsoft.com/quantum/quickstarts/install-python?tabs=tabid-conda) or [.NET](https://docs.microsoft.com/quantum/quickstarts/install-cs?tabs=tabid-cmdline%2Ctabid-csharp) language programs (C#, F#, etc.) for easy data processing and visualization
- [Jupyter notebooks](https://docs.microsoft.com/quantum/quickstarts/install-jupyter?tabs=tabid-conda) with either Q# or Python kernels


In [ ]:
%simulate UseCtlFunctor

## More than just simulation...

In [ ]:
%lsmagic

### Example target machine: Execution tracing!

In [ ]:
%trace Qrng

### Example target machine: Resource estimation!

In [ ]:
%estimate UseCtlFunctor

## Unit testing in Q#

- Great way to check that what we have typed matches paper results 😊

In [ ]:
open Microsoft.Quantum.Arrays;

operation AllocateQubitRegister(numQubits : Int) : Unit {

    Fact(numQubits > 0, "Expected a positive number.");
    
    using (register = Qubit[numQubits]) {
        ApplyToEach(AssertQubit(Zero, _), register);
    }
    Message("Test passed!");
}

In [ ]:
%simulate AllocateQubitRegister numQubits=5

## Unit testing in Q# cont.

- Helpful to test if qRAM optimizations still do the same thing
- Q# uses the _Choi–Jamiłkowski isomorphism_ to make an assertion of operation equivalence to one about preparing states
    <!--- We know that if you apply an operation and then it's adjoint, that should be an Identity operation
    - Make a channel that applies your operation under test, and then the adjoint of your reference operation.
    - If you start with a maximally entangled state, then apply the channel to one half of a maximally entangled state, then you can use state assertions to verify that you still have that same maximally entangled state.-->


In [ ]:
open Microsoft.Quantum.Diagnostics; 

operation ApplyCNOT(register : Qubit[])
: Unit is Adj + Ctl {
    CNOT(register[0], register[1]);
}

In [ ]:
operation ApplyCNOTTheOtherWay(register : Qubit[])
: Unit is Adj + Ctl {
    within {
        ApplyToEachCA(H, register);
    } 
    apply {
        CNOT(register[1], register[0]);
    }
}

operation CheckThatThisWorks() : Unit {
    AssertOperationsEqualReferenced(2, ApplyCNOT, ApplyCNOTTheOtherWay);
    Message("It works!");
}

In [ ]:
%trace CheckThatThisWorks --depth=4

## Our plan for qRAM:

🧰 **Step 1:** Use functions and operations to implement the qRAM proposals

📚 **Step 2:** Use libraries, functors, and other features to reduce how much code we need to write

🧪 **Step 3:** Use testing features to make sure our implementations are correct

💰 **Step 4:** Profit! (also publish 📃)

# Part 2: A case study of Q# driven research: The qRAM library

## Quantum applications _might_ need memory

- We need ways to transfer **classical data** to a **quantum system**
- _Some_ quantum algorithms, particularly quantum machine learning, assume access to a quantum RAM to load and store data during calculations.
- Queries at address $x$ can take many forms: 
    - Bit value as a phase (Grover's Algorithm): $\left|x\right\rangle\mapsto(-1)^{b_x}\left|x\right\rangle$
    - Bit value as a qubit (Element distinctness): $\left|x\right\rangle\left|0\right\rangle\mapsto\left|x\right\rangle\left|b_x\right\rangle$
    - Bit value as a complex vector of amplitudes (HHL Algorithm): $(b_0...b_n)\mapsto\sum\limits_{j} b_j\left|j\right\rangle$



# Quantum Memories (aka qRAM)

**Problem:** It is not clear if we will be able to do this eﬃciently at all, let alone in a fault-tolerant setting. 

 ❗ _An algorithmic speedup **may not** translate to an actual speedup in an application if it is not eﬃcient to use data in the ﬁrst place!_

😓 Physical limitations like coherence time, error rates, hardware supported gates, etc. contribute to the difficulty.

💡 There are many different approaches, each optimizing for a particular resource. 


## Deep Dive: qRAM approaches and tradeoffs:

#### http://bit.ly/between-the-bitlines

<center>
    <img src="media/olivia-talk-title.png" width="48%">

</center>

## We need:

- A way to implement many different algorithms representing qRAM proposals,
- count the resources needed for each proposal, and
- do it in an open and reproducible way.

<center>
    <img src="media/wanted.png" width="25%">
</center>


### https://github.com/qsharp-community/qram


<center>
    <img src="media/github-screencap.png" width="50%">
</center>

## Basic layout:

```
├───📃 docs 📃
├───🔮 samples 🔮
│   ├───BucketBrigade
│   ├───Grover
│   ├───Qrom
│   ├───ResourceEstimation
│   └───SelectSwap
├───✨ src ✨
└───🧪 tests 🧪
```

# `src`: where qRAMs are implemented
<center>
    <img src="media/src-screenshot.png" width="60%">
</center>

### Currently implemented proposals:
**qRAM**
- Bucket Brigade (bit and phase queries) [0708.1879](https://arxiv.org/abs/0708.1879)

**qROM**
- Simple [0807.4994](https://arxiv.org/abs/0807.4994)
- SELECTSWAP [1812.00954](https://arxiv.org/abs/1812.00954)

## Using a qROM

In [ ]:
%package QSharpCommunity.Libraries.Qram::0.1.35

## Libraries help us bootstrap

In [ ]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Measurement;
open Qram;

## Custom Types for quantum memories

```
newtype QRAM = (
    QueryPhase : ((AddressRegister, MemoryRegister, Qubit[]) => Unit is Adj + Ctl),
    QueryBit : ((AddressRegister, MemoryRegister, Qubit[]) => Unit is Adj + Ctl), 
    Write : ((MemoryRegister, MemoryCell) => Unit), 
    AddressSize : Int,
    DataSize : Int
);
```

## We can use function and custom types to handle classical data

In [ ]:
function ExampleMemoryData() : MemoryBank {
    let numDataBits = 3;
    let data =  [
        (0, IntAsBoolArray(0, numDataBits)), 
        (2, IntAsBoolArray(5, numDataBits)),
        (4, IntAsBoolArray(2, numDataBits)),
        (5, IntAsBoolArray(3, numDataBits))
    ];
    return GeneratedMemoryBank(Mapped(MemoryCell, data));
}

### Using a qROM (Read Only Memory)

In [ ]:
operation QueryQrom(queryAddress : Int) : Int {
    // Generate a (Int, Bool[]) array of data.
    let data = ExampleMemoryData();
    // Create the QRAM.
    let memory = QromOracle(data::DataSet);
    using ((addressRegister, targetRegister) = (Qubit[memory::AddressSize], Qubit[memory::DataSize])) {
        // Encode our query address
        ApplyPauliFromBitString (PauliX, true, IntAsBoolArray(queryAddress, memory::AddressSize), addressRegister);
        
        // Read from the memory
        memory::Read(LittleEndian(addressRegister), targetRegister);
        ResetAll(addressRegister);
        
        // Measure the target register as an integer
        return MeasureInteger(LittleEndian(targetRegister));
    }
}

### Using a qROM

```
// data: {(0, 0), (2, 5), (4, 2), (5, 3)}
```

In [ ]:
%simulate QueryQrom queryAddress=4

In [ ]:
%estimate QueryQrom queryAddress=2

# `tests`: ✔ our work
- Can run small instances on simulators
- Can verify resource counts on larger instances

<center>
    <img src="media/tests-screenshot.png" width="40%">
</center>


# What's next for the qRAM library?

📓 More documentation in an interactive browser

📄 Research paper compiling our results

❓ More qRAM/qROM proposals 

# Part 3: Quantum Intermediate Representation (QIR)

<center>
    <img src="media/qir.png" width="90%">
</center>

## QIR Spec

#### https://github.com/microsoft/qsharp-language/tree/main/Specifications/QIR
- **DRAFT** specification defines an intermediate representation for compiled quantum computations.
- Fork of LLVM, so it is easy for users to easily write code analyzers and code transformers that operate at this level, before the final target-specific code generation.
    - Cross language circuit optimization as a part of LLVM "pass" infrastructure
- It is neither required nor expected that any particular execution target actually implement every runtime function specified
    - Not limited to gate model QC





## QIR Projects ##

| Project Name | Description | Team | Link |
|--------------|-------------|------|------|
| DM-Sim       | DM-Sim is a density matrix quantum circuit simulator for multi-GPU accelerated HPC clusters. DIM-Sim is able to scale out to more than a thousand GPUs on ORNL Summit supercomputer, and is able to quickly simulate deep circuit with more than 1 million gates. DM-Sim supports Microsoft Q# through QIR, and IBM Qiskit & Google Cirq through OpenQASM. DIM-Sim provide easy-to-use Python API and C++ API. | Sriram Krishnamoorthy, Ang Li, Bo Fang | https://github.com/pnnl/DM-Sim/blob/master/doc/paper_sc20.pdf |
| QCOR         | QCOR is a C++ language extension and associated compiler implementation for heterogeneous quantum-classical computing. QCOR builds off the XACC framework and extends plugin interfaces from Clang to enable quantum kernel expression, compilation, and execution on currently available physical (and virtual) quantum backends. | Alex McCaskey, Thien Nguyen, Daniel Claudino, Anthony Santana, Hal Finkel, Tyler Kharazi, Dmitry Liakh | https://github.com/ornl-qci/qcor |

### Example syntax

In [ ]:
// Assumes that qb1 and qb2 are already in the |0> state
operation BellPair(qb1 : Qubit, qb2 : Qubit) : Unit
{
    H(qb1);
    CNOT(qb1, qb2);
}

```
define void @BellPair__body(%Qubit* %qb1, %Qubit* %qb2) {
entry:
  call void @__quantum__qis__h(%Qubit* %qb1)
  call void @__quantum__qis__cnot(%Qubit* %qb1, %Qubit* %qb2)
  ret void
}
```

## Demo: Q\# complier plugin that outputs QIR

https://github.com/microsoft/qsharp-compiler/blob/feature/qir/QIR-README.md

# 📝 Review 📝

- Q# is a high level, open-source language for writing _quantum algorithms_ in a portable and reproducable way ✔
- We have implemented a number of proposals for qRAMs in a Q# library and are working on resource estimation ✔
- QIR is a LLVM forked proposal for an intermediate representation of quantum programs ✔

## Want to start using Q#?

📑 [docs.microsoft.com/quantum](https://docs.microsoft.com/quantum)

📗 _Learn Quantum Computing with Python and Q#_: [bit.ly/qsharp-book](https://bit.ly/qsharp-book)

📺 Live quantum development on Twitch: [twitch.tv/crazy4pi314](https://twitch.tv/crazy4pi314)

🤝 Q# community group: [qsharp.community](https://qsharp.community)

🧩 Template repo for Q# projects complete with LaTeX paper setup [github.com/cgranade/quantum-research-template](https://github.com/cgranade/quantum-research-template)
